## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-25-18-53-39 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/25/us/win...
1,2026-01-25-18-53-33 +0000,nypost,Barack Obama issues rare political statement c...,https://nypost.com/2026/01/25/us-news/barack-o...
2,2026-01-25-18-51-10 +0000,bbc,Andy Burnham 'disappointed' after bid to becom...,https://www.bbc.com/news/articles/czr4e7ep468o...
3,2026-01-25-18-48-27 +0000,bbc,People cling to treetops as 'worst floods in a...,https://www.bbc.com/news/articles/cddgq9z4899o...
4,2026-01-25-18-48-02 +0000,startribune,Live: National Guard pulled back as questions ...,https://www.startribune.com/ice-raids-minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2409/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,shooting,25
48,minneapolis,22
16,trump,21
10,alex,18
11,pretti,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2026-01-25-14-56-34 +0000,nypost,Border Patrol chief Bovino praises agents in M...,https://nypost.com/2026/01/25/us-news/border-p...,103
1,2026-01-25-18-53-33 +0000,nypost,Barack Obama issues rare political statement c...,https://nypost.com/2026/01/25/us-news/barack-o...,101
93,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,101
47,2026-01-25-15-31-54 +0000,bbc,Video shows moments around fatal shooting of A...,https://www.bbc.com/news/videos/cp372pqq2rlo?a...,100
27,2026-01-25-16-46-09 +0000,nyt,"In Court Filings, Witnesses Describe Fatal Min...",https://www.nytimes.com/2026/01/24/us/witnesse...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,103,2026-01-25-14-56-34 +0000,nypost,Border Patrol chief Bovino praises agents in M...,https://nypost.com/2026/01/25/us-news/border-p...
167,51,2026-01-24-20-47-10 +0000,nypost,"Trilateral talks between US, Russia and Ukrain...",https://nypost.com/2026/01/24/world-news/trila...
6,44,2026-01-25-18-41-50 +0000,wapo,Live updates: Trump officials say Minneapolis ...,https://www.washingtonpost.com/nation/2026/01/...
34,42,2026-01-25-16-22-26 +0000,nypost,Here’s what airlines owe you if you’re strande...,https://nypost.com/2026/01/25/us-news/heres-wh...
165,32,2026-01-24-21-35-27 +0000,nypost,AOC urges US Senate to block ICE funding after...,https://nypost.com/2026/01/24/us-news/aoc-urge...
119,30,2026-01-25-04-44-43 +0000,nypost,Rock climber Alex Honnold reaches top of Taipe...,https://nypost.com/2026/01/24/lifestyle/rock-c...
59,29,2026-01-25-14-44-19 +0000,nypost,"NYC schools closed Monday due to snow storm, a...",https://nypost.com/2026/01/25/us-news/nyc-scho...
52,27,2026-01-25-15-11-58 +0000,wapo,Fatal shooting in Minneapolis renews protests;...,https://www.washingtonpost.com/nation/2026/01/...
43,22,2026-01-25-15-41-03 +0000,cbc,"No plans for China free trade deal, Carney say...",https://www.cbc.ca/news/politics/carney-canada...
26,20,2026-01-25-17-00-00 +0000,wsj,U.S. Speaker of the House Mike Johnson has hel...,https://www.wsj.com/politics/republicans-house...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
